In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import string
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import norm

In [2]:
# reading annual data
data = pd.read_excel('Returns_handbook_data.xls', 'Monthly')
data.head()

,Date (yyyymm),S&P 500 index,12-month moving sum of S&P 500 dividends,12-month moving sum of S&P 500 earnings,DJIA book-to-market value ratio,3-month Treasury bill yield (secondary market),Moodys AAA-rated corporate bond yield,Moodys BAA-rated corporate bond yield,Long-term government bond yield,Net equity expansion,Risk-free rate,CPI (all urban consumers) inflation rate,Long-term government bond return,Long-term corporate bond return,Monthly sum of squared daily returns on S&P 500 index,CRSP S&P 500 value-weighted return with dividends,CRSP S&P 500 value-weighted return excluding dividends,NBER recession dummies,NBER recession dummies with peak included
0,187101.0,4.44,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004955,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,187102.0,4.50,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,187103.0,4.61,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,187104.0,4.74,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,187105.0,4.86,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.003691,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# List of variables
# d12 -> dividend yeild
# e12 -> earning price ration 
# b/m => book to market
# tbl => Treasury bill rate  = -0.004 + 0.886 * Compercial paper rate
# default yield stpread = (AAA - BAA)
# lty => Long-term government bond yield
# ntis => net equity expansion 
# rfree => risk free rate
# infl => inflation
# ltr => long-term return
# corpr => corporate bond returns
# svar => stock variance
# csp => Cross sectional premiums 
# CRSP_SPww =>
# CRSP_SPvwx =>  
# tms => long term yield - trea

In [4]:
print('All column names')
print('--'*20)
for idx in range(len(data.columns)):
    print(idx, '--', string.ascii_lowercase[idx], '--', data.columns[idx])

All column names
----------------------------------------
0 -- a -- Date (yyyymm)
1 -- b -- S&P 500 index
2 -- c -- 12-month moving sum of S&P 500 dividends
3 -- d -- 12-month moving sum of S&P 500 earnings
4 -- e -- DJIA book-to-market value ratio
5 -- f -- 3-month Treasury bill yield (secondary market)
6 -- g -- Moodys AAA-rated corporate bond yield
7 -- h -- Moodys BAA-rated corporate bond yield
8 -- i -- Long-term government bond yield
9 -- j -- Net equity expansion
10 -- k -- Risk-free rate
11 -- l -- CPI (all urban consumers) inflation rate
12 -- m -- Long-term government bond return
13 -- n -- Long-term corporate bond return
14 -- o -- Monthly sum of squared daily returns on S&P 500 index
15 -- p -- CRSP S&P 500 value-weighted return with dividends
16 -- q -- CRSP S&P 500 value-weighted return excluding dividends
17 -- r -- NBER recession dummies
18 -- s -- NBER recession dummies with peak included


In [5]:
print('Shape of dataset:', data.shape)

Shape of dataset: (1682, 19)


In [27]:
# Equity premium

idx_first = 671 # starting data 1926:11
idx_last = 1679 # final date 2010:12

market_return = np.array(data.loc[idx_first:idx_last, ['CRSP S&P 500 value-weighted return with dividends']]).flatten()
r_f_lag = np.array(data.loc[(idx_first-1):(idx_last-1), 'Risk-free rate']) # risk free rate
equity_premium = np.log(1+market_return)-np.log(1+r_f_lag) # log excess return

In [28]:
#dividends
d12=np.array(data.loc[idx_first:idx_last, '12-month moving sum of S&P 500 dividends']) 
#S7P 500 index
sp500 = np.array(data.loc[idx_first:idx_last, 'S&P 500 index']) 
# log dividend-price ratio
dp = np.log(d12) - np.log(sp500) 
# S&P 500 index, lagged 
sp500_lag = np.array(data.loc[(idx_first-1):(idx_last-1), 'S&P 500 index']) # S&P 500 index, lagged 
# log dividend yield
dy = np.log(d12) - np.log(sp500_lag)
# earnings
e12 = np.array(data.loc[idx_first:idx_last, '12-month moving sum of S&P 500 earnings'])
# log earnings to price ratio
ep = np.log(e12) - np.log(sp500)
# log dividend-payout ratio
de = np.log(d12) - np.log(e12)
# volatility (SVAR)
svar = np.array(data.loc[idx_first:idx_last, 'Monthly sum of squared daily returns on S&P 500 index'])
# book to market ratio
bm = np.array(data.loc[idx_first:idx_last, 'DJIA book-to-market value ratio'])
# net equity issuing activity
ntis=np.array(data.loc[idx_first:idx_last, 'Net equity expansion'])
# t-bill raate
tbl = np.array(data.loc[idx_first:idx_last, '3-month Treasury bill yield (secondary market)'])
# long-term government bond yield
lty = np.array(data.loc[idx_first:idx_last, 'Long-term government bond yield'])
# long-term governmnent bond return
ltr = np.array(data.loc[idx_first:idx_last, 'Long-term government bond return'])
# term spread
tms=lty-tbl

# AAA-rated corporate bond yield
aaa = np.array(data.loc[idx_first:idx_last, 'Moodys AAA-rated corporate bond yield'])

# BAA-rated corporate bond yield
baa = np.array(data.loc[idx_first:idx_last, 'Moodys BAA-rated corporate bond yield'])

# Default yield spread
dfy = baa-aaa

# Long-term corporate bond return
corpr = np.array(data.loc[idx_first:idx_last, 'Long-term corporate bond return'])

# Default return spread
dfr = corpr - ltr

# Inflation, lagged(1926:11-2010:11)
infl_lag = np.array(data.loc[(idx_first-1):(idx_last-1), 'CPI (all urban consumers) inflation rate'])

In [29]:
# Combined arrays of all variable
econ = np.column_stack((dp, dy, ep, de, svar, bm, ntis, tbl, lty, ltr, tms, dfy, dfr, infl_lag))
econ_sink = np.column_stack((dp, dy, ep, svar, bm, ntis, tbl, lty, ltr, dfy, dfr, infl_lag))

In [30]:
# Sum of the parts variables

# earnings lagged (1926:11-2010:11)
e12_lag = np.array(data.loc[(idx_first-1):idx_last-1, '12-month moving sum of S&P 500 earnings'])
# earnings growth
e_growth = np.log((1/12)*e12) - np.log((1/12)*e12_lag) # earnings growth
# log (1+D/P)
dp_sop = np.log(1+(1/12)*(d12/sp500)) # log(1+D/P)
# risk-free rate
r_f = np.array(data.loc[idx_first:idx_last, 'Risk-free rate'])
# log risk free rate
r_f = np.log(1 + r_f)

In [31]:
# Estimating full-samples parameters for Campbell-Thompson restrictoins
beta_full = np.zeros((econ.shape[1], 1))
T = equity_premium.shape[0]
lhs = equity_premium[1: T]

i = 0
for i in range(econ.shape[1]):
    rhs_i = np.column_stack((econ[:T-1, i], np.ones(T-1)))
    model = sm.OLS(lhs, rhs_i)
    results_i = model.fit()
    beta_full[i] = results_i.params[0]

# the fifth column is svar.
beta_full[4] = 1 # restricting SVAR slope coeeficient to be positive
print(beta_full)

[[ 0.00647033]
 [ 0.00781829]
 [ 0.00864035]
 [-0.00233856]
 [ 1.        ]
 [ 0.01514282]
 [-0.13465435]
 [-0.07879868]
 [-0.05998025]
 [ 0.1030293 ]
 [ 0.15664405]
 [ 0.09652975]
 [ 0.17803721]
 [-0.19076655]]


In [32]:
# preliminaries

Y = equity_premium
T = Y.shape[0]
N = econ.shape[1]
R = (1946-1926)*12+1 # In-sample period from 1926:12-1946:12
P_0 = (1956-1946)*12 # holdout out-of-sample period, 1947:01 - 1956:12
P = T - (R+P_0) # forecast evaluaiton period, 1957:01 - 2010:12
theta = 0.75 # discount factor for DMSFE pooled forecast
r = 1 # Number of principal components
MA_SOP = 20*12 # window size for each forecast

## Computing forecasts, 1947:01-2010:12 (include holdout OOS period)

In [294]:
# Computing forecasts, 1947:01-2010:12 (include holdout OOS period)
FC_HA=np.zeros((P_0+P,1));
FC_ECON=np.zeros((P_0+P,N));
beta_ECON=np.zeros((P_0+P,N,2));
FC_ECON_CT=np.zeros((P_0+P,N));
omega_DMSFE=np.zeros((P_0+P,N));
FC_OTHER=np.zeros((P_0+P,6));
FC_OTHER_CT=np.zeros((P_0+P,6));

for t in range(P_0+P):
    FC_HA[t] = np.mean(Y[1:(R+(t-1))])
    X_t = econ[:R+(t-1)-1, :]
    Y_t = Y[1:R+(t-1)]
    
    # individual regression
    for i in range(N):
        try:
            model = sm.OLS(Y_t, np.column_stack((X_t[:, i], np.ones(R+(t-1)-1))))
            results_t_i = model.fit()
        except:
            print('Error Fitting model')
            break

        # performing a 2*1 and 1*2 matrix
        FC_ECON[t, i] = econ[R+(t-1),i] * results_t_i.params[0] + 1.0 * results_t_i.params[1]
        # beta => coefficient
        beta_ECON[t, i, 0] = results_t_i.params[0]
        # bse => standard error
        beta_ECON[t, i, 1] = results_t_i.bse[0]

        if beta_full[i] > 0:
            if results_t_i.params[0] > 0:
                FC_ECON_CT[t, i] = FC_ECON[t, i]
            elif results_t_i.params[0] < 0:
                FC_ECON_CT[t, i] = FC_HA[t]

        elif beta_full[i] < 0:
            if results_t_i.params[0] < 0:
                FC_ECON_CT[t, i] = FC_ECON[t, i]
            elif results_t_i.params[0] > 0:
                FC_ECON_CT[t, i] = FC_HA[t]

        if FC_ECON_CT[t, i] < 0:
            FC_ECON_CT[t, i] = 0

    if t > P_0:

        # Kitchen sink forecast
        X_t_sink = econ_sink[:R+(t-1)-1, :]
        model = sm.OLS(Y_t, np.column_stack((X_t_sink, np.ones(R+(t-1)-1))))
        results_t_sink = model.fit()
        FC_OTHER[t, 0] = np.dot(np.append(econ_sink[R+(t-1), :], 1.0), results_t_sink.params)
        if FC_OTHER[t, 0] < 0:
            FC_OTHER_CT[t, 0] = 0
        else:
            FC_OTHER_CT[t, 0] = FC_OTHER[t, 0]
        
        # Pooled forecast: simple average
        FC_OTHER[t, 2] = np.mean(FC_ECON[t, :])
        # if value is negative, we change it to zero.
        if FC_OTHER[t, 2] < 0:
            FC_OTHER_CT[t, 2] = 0
        else:
            FC_OTHER_CT[t, 2] = FC_OTHER[t, 2]

        # Pooled forecast: DMSFE
        powers_t = np.array(range(t-2, -1, -1))
        # solving for
        # m=sum((kron(ones(1,N),(theta*ones(t-1,1)).^powers_t)).*((kron(ones(1,N),Y(R+1:R+(t-1)))-FC_ECON(1:(t-1),:)).^2))';
        theta_powered_up = (np.power(theta*np.ones((t-1)), powers_t))
        kron_product_power = np.kron(np.ones((1, N)), theta_powered_up.reshape((t-1, 1)))
        kron_product_Y = np.kron(np.ones((1, N)), Y[R: R+(t-1)].reshape((t-1,1)))
        part_2 = np.power(kron_product_Y - FC_ECON[:(t-1),:], 2)
        m = np.sum(kron_product_power * part_2, axis=0)
        omega = (np.power(m, -1))/(np.sum(np.power(m, -1)))
        omega_DMSFE[t, :] = omega
        FC_OTHER[t, 3] = np.dot(FC_ECON[t, :], omega)
        if FC_OTHER[t, 3] < 0:
            FC_OTHER_CT[t, 3] = 0
        else:
            FC_OTHER_CT[t, 3] = FC_OTHER[t, 3]
                 
        # Sum of the parts forecast
        FC_OTHER[t, 5] = np.mean(e_growth[R+(t-1)-MA_SOP+1:R+(t-1)]) + dp_sop[R+(t-1)] - r_f[R+(t-1)]
        if FC_OTHER[t, 5] < 0:
            FC_OTHER_CT[t, 5] = 0
        else:
            FC_OTHER_CT[t, 5] = FC_OTHER[t, 5]

## Forecast Evaluation

In [295]:
## Forecast evaluation based on MSFE, 1957:01 - 2010:12

beta_ECON = beta_ECON[P_0+1:, :, :]
actual = np.reshape(Y[R+P_0+1:], (Y[R+P_0+1:].shape[0], 1))
FC_HA = FC_HA[P_0+1:]
FC_ECON = FC_ECON[P_0+1:, :]
FC_ECON_CT = FC_ECON_CT[P_0+1:, :]
FC_OTHER = FC_OTHER[P_0+1:, :]
FC_OTHER_CT = FC_OTHER_CT[P_0+1:, :]
omega_DMSFE = omega_DMSFE[P_0+1:, :]

In [296]:
'''
    Evaluating for individual predictors
'''
e_HA = actual - FC_HA
e_ECON = np.kron(np.ones((1, FC_ECON.shape[1])), actual) - FC_ECON
e_ECON_CT = np.kron(np.ones((1, FC_ECON.shape[1])), actual) - FC_ECON_CT
CSFE_HA = np.cumsum(np.power(e_HA, 2))
CSFE_ECON = np.cumsum(np.power(e_ECON, 2))
CSFE_ECON_CT = np.cumsum(np.power(e_ECON_CT, 2))
DCSFE_ECON = np.kron(np.ones((1, FC_ECON.shape[1])), CSFE_HA) - CSFE_ECON
DCSFE_ECON_CT = np.kron(np.ones((1, FC_ECON_CT.shape[1])), CSFE_HA) - CSFE_ECON_CT
R2OS_ECON=np.zeros((FC_ECON.shape[1],6));
R2OS_ECON_CT=np.zeros((FC_ECON.shape[1], 6));

for i in range(R2OS_ECON.shape[0]):

    # Over all
    
    # Without restriction
    R2OS_ECON[i, 0] = 100*(1- (np.sum(np.power(e_ECON[:, i], 2)) / np.sum(np.power(e_HA, 2))))
    e_ECON_reshaped = e_ECON[:, i].reshape(e_ECON.shape[0], 1)
    FC_ECON_reshaped = FC_ECON[:, i].reshape(FC_ECON.shape[0], 1)
    f_i = np.power(e_HA, 2) - (np.power(e_ECON_reshaped,2) - np.power(FC_HA - FC_ECON_reshaped, 2))
    results_i  = sm.OLS(f_i, np.ones((f_i.shape[0], 1))).fit()
    R2OS_ECON[i, 1] = 1 - norm.cdf(results_i.tvalues[0], loc=0, scale=1)
    
    # With Campbell thompson restriction
    R2OS_ECON_CT[i, 0] = 100*(1- (np.sum(np.power(e_ECON_CT[:, i], 2)) / np.sum(np.power(e_HA, 2))))
    e_ECON_CT_reshaped = e_ECON_CT[:, i].reshape(e_ECON_CT.shape[0], 1)
    FC_ECON_CT_reshaped = FC_ECON_CT[:, i].reshape(FC_ECON_CT.shape[0], 1)
    f_i = np.power(e_HA, 2) - (np.power(e_ECON_CT_reshaped,2) - np.power(FC_HA - FC_ECON_CT_reshaped, 2))
    results_i  = sm.OLS(f_i, np.ones((f_i.shape[0], 1))).fit()
    R2OS_ECON_CT[i, 1] = 1 - norm.cdf(results_i.tvalues[0], loc=0, scale=1)


In [297]:
'''
    Evaluating for combined predictors
'''

# For combined variables
e_OTHER = np.kron(np.ones((1, FC_OTHER.shape[1])), actual) - FC_OTHER
e_OTHER_CT = np.kron(np.ones((1, FC_OTHER_CT.shape[1])), actual) - FC_OTHER_CT
CSFE_OTHER = np.cumsum(np.power(e_OTHER, 2))
CSFE_OTHER_CT = np.cumsum(np.power(e_OTHER_CT,2))
DCSFE_OTHER = np.kron(np.ones((1, FC_OTHER.shape[1])), CSFE_HA) - CSFE_OTHER
DCSFE_OTHER_CT = np.kron(np.ones((1, FC_OTHER_CT.shape[1])), CSFE_HA) - CSFE_OTHER_CT
R2OS_OTHER = np.zeros((FC_OTHER.shape[1], 6))
R2OS_OTHER_CT = np.zeros((FC_OTHER.shape[1], 6))

for i in range(R2OS_OTHER.shape[0]):

    # Without restriction
    R2OS_OTHER[i, 0] = 100*(1- (np.sum(np.power(e_OTHER[:, i], 2)) / np.sum(np.power(e_HA, 2))))
    e_OTHER_reshaped = e_OTHER[:, i].reshape(e_OTHER.shape[0], 1)
    FC_OTHER_reshaped = FC_OTHER[:, i].reshape(FC_OTHER.shape[0], 1)
    f_i = np.power(e_HA, 2) - (np.power(e_OTHER_reshaped,2) - np.power(FC_HA - FC_OTHER_reshaped, 2)) 
    results_i  = sm.OLS(f_i, np.ones((f_i.shape[0], 1))).fit()
    R2OS_OTHER[i, 1] = 1 - norm.cdf(results_i.tvalues[0], loc=0, scale=1)
    
    # Without restriction
    R2OS_OTHER_CT[i, 0] = 100*(1- (np.sum(np.power(e_OTHER_CT[:, i], 2)) / np.sum(np.power(e_HA, 2))))
    e_OTHER_CT_reshaped = e_OTHER_CT[:, i].reshape(e_OTHER_CT.shape[0], 1)
    FC_OTHER_CT_reshaped = FC_OTHER_CT[:, i].reshape(FC_OTHER_CT.shape[0], 1)
    f_i = np.power(e_HA, 2) - (np.power(e_OTHER_CT_reshaped,2) - np.power(FC_HA - FC_OTHER_CT_reshaped, 2)) 
    results_i  = sm.OLS(f_i, np.ones((f_i.shape[0], 1))).fit()
    R2OS_OTHER_CT[i, 1] = 1 - norm.cdf(results_i.tvalues[0], loc=0, scale=1)

In [298]:
predictor_labels = ['log(DP)','log(DY)','log(EP)','log(DE)','SVAR',
                    'BM','NTIS','TBL','LTY','LTR','TMS','DFY','DFR','INFL']

predictor_labels_combined = ['Kitchen sink','SIC','POOL-AVG',
                             'POOL-DMSFE','Diffusion index','Sum-of-the-parts']

# predictor_labels_combined = ['Kitchen sink','POOL-AVG','POOL-DMSFE']

In [299]:
# unrestricted overall
print('Results for Individual predictors (unrestricted)')
results_ind_overall = pd.DataFrame()
results_ind_overall['predictors'] = predictor_labels
results_ind_overall['R2OS'] = R2OS_ECON[:,0]
results_ind_overall['p-value'] = R2OS_ECON[:,1]
results_ind_overall

Results for Individual predictors (unrestricted)


,predictors,R2OS,p-value
0,log(DP),-0.117797,0.108982
1,log(DY),-0.466442,0.077922
2,log(EP),-2.048954,0.310191
3,log(DE),-2.114508,0.974457
4,SVAR,0.252428,0.191416
5,BM,-1.817668,0.326423
6,NTIS,-0.990026,0.437150
7,TBL,-0.024163,0.089048
8,LTY,-1.312167,0.131826
9,LTR,0.005099,0.170533


In [306]:
print('Results for Individual predictors (Campbell Thompson Restrictions)')
results_ind_ct_overall = pd.DataFrame()
results_ind_ct_overall['predictors'] = predictor_labels
results_ind_ct_overall['R2OS'] = R2OS_ECON_CT[:,0]
results_ind_ct_overall['p-value'] = R2OS_ECON_CT[:,1]
results_ind_ct_overall

Results for Individual predictors (Campbell Thompson Restrictions)


,predictors,R2OS,p-value
0,log(DP),0.103733,0.077219
1,log(DY),0.119368,0.049820
2,log(EP),-0.830655,0.248949
3,log(DE),-1.694413,0.966191
4,SVAR,0.000000,NaN
5,BM,-1.222192,0.307254
6,NTIS,-0.987248,0.436449
7,TBL,0.168297,0.114893
8,LTY,-0.079489,0.109769
9,LTR,0.259379,0.107366


In [304]:
# unrestricted overall
print('Results for Combined predictors (unrestricted)')
results_overall = pd.DataFrame()
results_overall['predictors'] = predictor_labels_combined
results_overall['R2OS'] = R2OS_OTHER[:,0]
results_overall['p-value'] = R2OS_OTHER[:,1]

# Only showing frist four predictors as others haven't been completed
results_overall.iloc[[0,2,3,5],:]

Results for Combined predictors (unrestricted)


,predictors,R2OS,p-value
0,Kitchen sink,-8.939185,0.486110
2,POOL-AVG,0.392930,0.034129
3,POOL-DMSFE,0.459226,0.024745
5,Sum-of-the-parts,0.861341,0.010507


In [305]:
print('Results for Combined predictors (Campbell Thompson Restrictions)')
results_overall_ct = pd.DataFrame()
results_overall_ct['predictors'] = predictor_labels_combined
results_overall_ct['R2OS'] = R2OS_OTHER_CT[:,0]
results_overall_ct['p-value'] = R2OS_OTHER_CT[:,1]
results_overall_ct.iloc[[0,2,3,5],:]

Results for Combined predictors (Campbell Thompson Restrictions)


,predictors,R2OS,p-value
0,Kitchen sink,-2.194704,0.499963
2,POOL-AVG,0.400734,0.031513
3,POOL-DMSFE,0.478735,0.019931
5,Sum-of-the-parts,0.914883,0.007207
